In [1]:
import pandas as pd
import numpy as np
import geopy.distance
import math

In [2]:
#routine for calculating the angle of the transmission antenna.
def get_bearing(lat2,lon2):
    dLon = lon2 - (-51.794696); #calculating the change in longitude.
    y = math.sin(dLon) * math.cos(lat2);
    x = math.cos(-30.847095)*math.sin(lat2) - math.sin(-30.847095)*math.cos(lat2)*math.cos(dLon); # This vector represents the direction from a fixed point (latitude -30.847095, longitude -51.794696) to the given coordinates (lat2, lon2).
    brng = np.rad2deg(math.atan2(y, x)); #This line computes the bearing angle using the atan2 function. The atan2 function returns the angle whose tangent is the quotient of the specified y and x values. np.rad2deg is used to convert the result from radians to degrees.
    if brng < 0: brng+= 360 #This conditional statement checks if the calculated bearing (brng) is less than 0. If true, it adds 360 degrees to the bearing. This step ensures that the bearing is represented as a positive angle, and it falls within the range [0, 360).
    brng=round(brng) #The round function is applied to round the bearing angle to the nearest integer
    return brng # the function returns the calculated and processed bearing angle.


In [3]:

#routine for calculating antenna gain
def get_gain(angle):
    a=angle+az; #adjusting the angle by adding another value (az), which might represent an azimuth or direction.
    if a > 359: a = a - 360 #ensure that the angle stays within the range [0, 359].
    G=dant.iloc[a, 1] #Here, dant appears to be a DataFrame or another data structure with two-dimensional indexing. The variable G is assigned the value found in the DataFrame dant at row a and column index 1. This line retrieves a gain value associated with the adjusted angle.
    return G


In [4]:
# Read file
df = pd.read_csv('Dataset1.txt', delimiter=',') #This line reads a CSV file named 'Sel_Cama_PCI320_(900).csv' into a Pandas DataFrame (df). The data in the CSV file is assumed to be comma-separated, as indicated by the delimiter=',' parameter.

df = df[['Lon', 'Lat','RSRP','Region']]    #This line selects specific columns ('Lon', 'Lat', 'RSRP', 'Regiao') from the DataFrame df and assigns the result back to df. This step filters the DataFrame to include only the relevant columns for further analysis.

dant = pd.read_csv('Dataset2.txt', delimiter=',') #This line reads another CSV file named 'Antena_cama.csv' into a new Pandas DataFrame (dant). In this case, the data in the CSV file is assumed to be semicolon-separated, as indicated by the delimiter=';' parameter.


In [5]:
#transmitter coordinates
p1 = (-30.847095, -51.794696)#This line defines a tuple p1 representing coordinates (-30.847095, -51.794696), which seems to be the coordinates for a location named 'Camaqua'.
az = 120 #This line assigns the value 120 to the variable az, which is likely representing the azimuth or direction associated with the 'Camaqua' location.

#calculates distances and angles
dist=[]
bearing=[]
att=[]
Gain=[]


In [6]:
for _, pontos in df.iterrows(): #This line initiates a loop that iterates through the rows of the DataFrame df using iterrows(). The loop variable pontos will represent each row of the DataFrame in each iteration.
    p2=[pontos['Lat'], pontos['Lon']] #Within the loop, this line creates a list p2 containing the latitude and longitude values from the current row (pontos) of the DataFrame. This represents the coordinates of a point in the DataFrame.
    dist.append((geopy.distance.distance(p1, p2).km)*1000) #In this line, the code calculates the distance between the two points p1 and p2 using the geopy.distance.distance function. The result is in kilometers, so it's multiplied by 1000 to convert it to meters. The calculated distance is then appended to the dist list.
    bearing.append(get_bearing(pontos['Lat'], pontos['Lon'])) #This line calculates the bearing angle between the two points using a function called get_bearing. The latitude and longitude values from the current row (pontos) are passed as arguments to this function. The resulting bearing angle is then appended to the bearing list.



In [7]:
df["Distance"] = dist
df["Bearing"] = bearing
#After the loop, these lines add two new columns, "distancia" and "bearing", to the DataFrame df.



In [8]:
for i in range(len(df)):
    ang = df.iloc[i, 5]
    adjusted_angle = (ang + az) % 360  # Use modulo operator to keep the angle in [0, 359]

    try:
        G = dant.iloc[adjusted_angle, 1]
        Gain.append(G)
    except IndexError:
        # Handle the case where the index is out of bounds
        Gain.append(None)  # Or you can assign a default value


In [9]:
df["Gain"] = Gain #After the loop, this line adds a new column named "Ganho" to the DataFrame df and populates it with the values from the Gant list. This column likely contains gain values associated with the angles.

df=df.sort_values(by=['Distance']) #This line sorts the DataFrame df based on the values in the "distancia" column in ascending order. This step organizes the DataFrame by distance, so the points are ordered from the closest to the reference point to the farthest.

df.to_csv('NEW CALC.csv') #Finally, this line writes the sorted DataFrame df to a new CSV file named 'Sel_Cama_PCI320_(900)_dis.csv'. This file likely contains the original data with additional columns for distance, bearing, gain, and has been sorted by distance.

print("End")

End


## 